# **SQL-4. Сложные объединения** 

***
#### **ЗАДАЧИ**:

Укажите название города с максимальным весом единичной доставки.

select  
   sh.*,  
   ci.city_name  
from    
    sql.shipment sh  
join  
    sql.city ci on sh.city_id = ci.city_id  
order by  
    weight desc  

Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

select  
    dr.first_name,  
    sh.cust_id,  
    count(sh.cust_id) ship_amount  
from  
    sql.shipment sh  
join  
    sql.driver dr  on dr.driver_id = sh.driver_id  
group by  
    dr.driver_id, sh.cust_id  
order by  
    ship_amount desc  

Укажите даты первой и последней по времени доставок в таблице shipment

select  
    *  
from  
    sql.shipment  
order by   
    ship_date desc  

Укажите имя клиента, получившего наибольшее количество доставок за 2017 год

select  
    cu.cust_name,    
    count(sh.ship_id) ship_amount   
from    
    sql.shipment sh   
join  
    sql.customer cu on sh.cust_id = cu.cust_id  
where  
    ship_date>'December 31, 2016'  
group by  
    cu.cust_id  
order by  
    ship_amount desc  

***
#### **UNION**

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

SELECT book_name object_name, 'книга' object_descritption /*выбираем столбец с названием book_name, задаём алиас для столбца object_name,  задаём во второй колонке объект ‘книга’ с алиасом для столбца object_descritption*/  
FROM public.books /*из схемы public и таблицы books*/  
UNION ALL /*оператор присоединения*/  
SELECT movie_title, 'фильм' /*выбираем столбец movie_title, сами задаём во второй колонке объект ‘фильм’*/  
FROM sql.kinopoisk /*из схемы sql и таблицы kinopoisk*/  

Визуально произведённое нами действие можно представить следующим образом:

![visual](https://lms.skillfactory.ru/assets/courseware/v1/7e7950b737303d748fd0b38616e377d8/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_2_1.png)

В запросе мы использовали оператор **UNION ALL** — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:
* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.
***  
##### **ВИДЫ UNION**

Оператор присоединения существует в двух вариантах:

**UNION** выводит только уникальные записи;  
**UNION ALL** присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.  

Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.
***
##### **СИНТАКСИС**
Запрос строится таким образом:

SELECT  
         n columns  
FROM   
         table_1  
  
UNION ALL  
  
SELECT   
         n columns  
FROM   
         table_2  
...   
   
UNION ALL  

SELECT   
         n columns  
FROM   
         table_n  
         
Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.

Важно! **Названия** итоговых колонок в выводе будут **такие же, как в первом блоке SELECT**, даже если они отличаются в других блоках подзапросов.

Другая особенность — в применении **сортировки ORDER BY**: она всегда будет **относиться к итоговому результату всего запроса** с UNION ALL.

В случаях, когда необходимо применить команду ORDER BY или LIMIT **не к итоговому** результату, а к каждой части запроса, можно **обернуть подзапросы в скобки**.
Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.  
А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.  
Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!  

(SELECT book_name object_name, 'книга' object_descritption   
FROM public.books  
ORDER BY 1  
LIMIT 1)  
UNION ALL  
(SELECT movie_title, 'фильм'   
FROM sql.kinopoisk  
ORDER BY 1  
LIMIT 1)  
***
##### **ЗАДАЧИ**

Напишите запрос, который создаёт уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков.  
Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck).  
Отсортируйте список по названию объекта, а затем — по типу.  

select  
    c.city_name object_name,  
    'city' object_type  
from  
    sql.city c  
union  
select  
    c.state,  
    'state' object_type  
from  
    sql.city c  
union  
select  
    d.first_name,  
    'driver' object_type  
from  
    sql.driver d  
union  
select   
    t.make,  
    'truck' object_type  
from  
    sql.truck t  
order by  
    object_name, object_type  

Напишите запрос, который соберёт имена всех упомянутых городов и штатов с таблицы city.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

select  
    c.city_name object_name
from  
    sql.city c  
union all  
select  
    c.state 
from  
    sql.city c  
order by  
    object_name  

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов.
Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

*то же, но без ALL*

***
#### **UNION и ограничение типов данных**

Как мы уже знаем, UNION может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.  
Допустим, мы хотим вывести список всех id городов и их названий в одном столбце.  
Вместо результата вы получите сообщение об ошибке: *"ERROR: UNION types integer and text cannot be matched"*. Дело в том, что мы попытались объединить числовой и строковый типы в одной колонке, а это невозможно.  
Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.  

Для типизации в Postgres составляется запрос по модели 

**column_name::column_type**.

Таким образом, **чтобы перевести city_id в текст**, нам потребуется написать **city_id::text**.  
Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.  

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник. Также добавьте столбец с именем водителя и столбец с типом контакта (phone или zip в зависимости от типа). Столбцы к выводу: contact, first_name, contact_type.
Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

select  
    d.zip_code::text contact,  
    d.first_name::text first_name,  
    'zip' contact_type  
from  
    sql.driver d  
union all   
select  
    d.phone::text contact,  
    d.first_name::text first_name,  
    'phone' contact_type  
from  
    sql.driver d  
order by
    contact, first_name

***
#### **ВОЗМОЖНОСТИ UNION**
Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.

SELECT  
         c.city_name,  
         c.population /*выбираем столбцы city_name, population*/  
FROM  
         sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/  

UNION ALL /*оператор присоединения*/  

SELECT  
         'total',  
         SUM(c.population) /*сами задаём объект ‘total’, суммируем все значения столбца population*/  
FROM  
         sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/  
ORDER BY 2 DESC /*сортируем по второму столбцу в убывающем порядке (чтобы итоговая сумма была в начале)*/  

![visual](https://lms.skillfactory.ru/assets/courseware/v1/5b38c2b9fa661c3e2c559506101f4fb7/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_4_1.png)

ЗАДАЧА:

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipping.
Не забывайте о единой типизации.
Упорядочьте по убыванию столбца date_period.

SELECT  
    s.ship_date::text date_period,  
    count(s.ship_id) cnt_shipping  
FROM  
    sql.shipment s  
group BY    
    date_period  
UNION ALL  
SELECT  
    'total_shipments',  
    count(s.ship_id)  
FROM  
    sql.shipment s  
ORDER BY 1 DESC  

***
#### **UNION и дополнительные условия**

UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

Например, с помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона.

SELECT  
         d.first_name,  
         d.last_name,  
         'телефон заполнен' phone_info  
FROM  
         sql.driver d  
WHERE d.phone IS NOT NULL  

UNION  

SELECT  
         d.first_name,  
         d.last_name,  
         'телефон не заполнен' phone_info  
FROM  
         sql.driver d  
WHERE d.phone IS NULL  

Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:  
* если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
* если нет — выводим 'доставка не осуществлялась'.
Столбцы к выводу: city_name, state, shipping_status.  
Отсортируйте в алфавитном порядке по городу, а затем — по штату.  

SELECT  
    c.city_name city_name,  
    c.state state,  
    'доставка осуществлялась' shipping_status  
FROM  
    sql.city c  
left join  
    sql.shipment s on s.city_id = c.city_id  
WHERE s.city_id IS NOT NULL  
UNION  
SELECT  
    c.city_name city_name,  
    c.state state,  
    'доставка не осуществлялась' shipping_status  
FROM  
    sql.city c  
left join  
    sql.shipment s on s.city_id = c.city_id  
WHERE s.city_id IS NULL  
order by  
    city_name, state  

Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки.
Пусть первый столбец содержит название города, а второй формируется так:
* если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
* иначе — вывести количество доставок, увеличенное на пять.
Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

SELECT  
    c.city_name city_name,  
    count(s.city_id) shippings_fake  
FROM  
    sql.city c  
join  
    sql.shipment s on s.city_id = c.city_id  
group by  
    c.city_name, c.state  
having  
    count(s.city_id)>10  
UNION  
SELECT   
    c.city_name city_name,  
    count(s.city_id)+5 shippings_fake  
FROM  
    sql.city c  
join  
    sql.shipment s on s.city_id = c.city_id  
group by  
    c.city_name, c.state  
having  
    count(s.city_id)<=10   
order by   
    shippings_fake desc, city_name   



***
#### **UNION и ручная генерация**

Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.

SELECT  
         'a' letter,'1' ordinal_position /*сами задаём значение первого столбца ‘a’ и алиас для него letter, значение второго столбца ‘1’ и алиас для него ordinal_position*/  
         
UNION /*оператор присоединения*/  

SELECT   
         'b','2' /*сами задаём значение первого столбца ‘b’, значение второго столбца ‘2’ */  
         
UNION /*оператор присоединения*/  
 
SELECT  
         'c','3' /*сами задаём значение первого столбца ‘с’, значение второго столбца ‘3’*/   

Напишите запрос, который выберет наибольшее из значений:
* 1000000;
* 541;
* -500;
* 100.

SELECT   
    '1000000'::int number  
UNION   
SELECT   
    '541'  
UNION   
SELECT  
    '-500'  
UNION   
SELECT  
    '100'  
order by number desc  
limit 1  

***
#### **EXCEPT**

ИСКЛЮЧАЕМ ПОВТОРЯЮЩИЕСЯ ДАННЫЕ

![EXCEPT](https://lms.skillfactory.ru/assets/courseware/v1/fd51ef1b05d2fdc6f1c2e860cffb4cd7/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_7_1.png)

Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.

SELECT  
         c.city_name  
FROM  
         sql.shipment s  
JOIN sql.city c ON s.city_id = c.city_id   

**EXCEPT**  

SELECT  
         cc.city_name  
FROM  
         sql.driver d   
JOIN sql.city cc ON d.city_id=cc.city_id  
ORDER BY 1  

Синтаксические правила для оператора EXCEPT такие же, как и для UNION:

* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

Мы уже знаем, как решить такую задачу с использованием **LEFT JOIN**. Вариант с **EXCEPT** будет полезен в тех случаях, когда у вас **много столбцов** и вам не хочется прописывать их равенство в условии для JOIN.

Выведите список zip-кодов, которые **есть в первой таблице** (sql.driver), **но отсутствуют во второй таблице** (sql.customer). Отсортируйте по возрастанию, столбец к выводу — zip.

select  
    d.zip_code zip  
from  
    sql.driver d  
except  
select  
    cu.zip zip  
from  
    sql.customer cu  
order by  
    zip  

***
#### **INTERSECT**

ВЫБИРАЕМ ОБЩИЕ ДАННЫЕ

![INTERSECT](https://lms.skillfactory.ru/assets/courseware/v1/5c4422f466caff0a493c3ec664658ae8/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_8_1.png)

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах?

Предположим, нам надо вывести совпадающие по названию города и штаты.

SELECT   
         c.city_name object_name  
FROM   
         sql.city c  

**INTERSECT**

SELECT   
         cc.state  
FROM   
         sql.city cc  
ORDER BY 1  

Как видим, с помощью оператора INTERSECT мы вывели названия городов и штатов, которые совпадают: New York, Washington и Wyoming.
Оператор INTERSECT оставляет только те строки, которые являются **общими для двух запросов**.  
**Как EXCEPT, так и INTERSECT убирают дубликаты, если они имеются.**  

Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.

SELECT   
    c.city_id id   
FROM   
    sql.city c  

INTERSECT  

SELECT   
    d.city_id  
FROM   
    sql.driver d  

INTERSECT  

SELECT   
    cu.city_id  
FROM   
    sql.customer cu  
    
INTERSECT  

SELECT   
    s.city_id  
FROM   
    sql.shipment s  

Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.

SELECT   
    cu.zip zip  
FROM   
    sql.customer cu  

INTERSECT  

SELECT   
    d.zip_code   
FROM   
    sql.driver d  

***
#### **ЗАДАЧИ**

* Выведите города с максимальным и минимальным весом единичной доставки.
Столбцы к выводу — city_name, weight.

(select  
    c.city_name,  
    max(s.weight) weight  
from  
    sql.shipment s  
left join sql.city c on s.city_id = c.city_id  
group by  
    c.city_name   
order by  
    weight desc   
limit 1)  
    
union  

(select  
    c.city_name,  
    min(s.weight) weight  
from  
    sql.shipment s  
left join sql.city c on s.city_id = c.city_id  
group by  
    c.city_name  
order by  
    weight  
limit 1)  
  
* Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id).
Столбец к выводу — mutual_id.
Отсортируйте по возрастанию.

select  
    s.cust_id mutual_id  
from  
    sql.shipment s  

intersect  

select  
    s.ship_id mutual_id  
from  
    sql.shipment s  
order by  
    mutual_id  

* Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'.
Столбцы к выводу — object_name, object_description.
Отсортируйте по названию в алфавитном порядке.

select distinct  
    cu.cust_name object_name,  
    'КЛИЕНТ' object_description  
from  
    sql.customer cu  
where  
    cust_type='manufacturer'  

union  

select distinct   
    t.make object_name,  
    'ГРУЗОВИК' object_description  
from  
    sql.truck t  

order by  
    object_name  
